In [59]:
import torch
import torch.nn as nn
import json
import numpy as np
from model import Seq2Seq
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from data_loader import get_loader

In [60]:
with open('./data/word2id.json', 'r') as f:
    word2id = json.load(f)
id2word = {i: w for w, i in word2id.items()}
src_root = './data/task1/source/'
trg_root = './data/task1/target/'
data_loader = get_loader(src_root, trg_root, 10, num_workers=2)

In [61]:
model_save_path = './data/model.pth'
vocab_size = 100
model = Seq2Seq(vocab_size)
model.cuda()
model.load_state_dict(torch.load(model_save_path))

In [62]:
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

In [65]:
for i, (sources, targets, source_lengths, target_lengths) in enumerate(data_loader):
    
    sources = to_var(sources)
    targets_out = targets[:, 1:]
    max_length = targets_out.size(1)
    
    # Forward pass
    h, c = model.encode(sources, source_lengths)
    batch_size = sources.size(0)
    input = to_var(torch.LongTensor([word2id['<SOS>']] * batch_size)).unsqueeze(1)  # (batch_size, 1)
    output = model.sample(input, h, c, max_length)
    
    
    
    # Compute accuracy
    
    
    if i == 1:   # should be deleted
        break

In [66]:
mask = (targets_out != 0).float()
correct = (output.data.cpu() == targets_out).float() * mask  # (batch_size, max_length)
mask_sum = torch.sum(mask, dim=1)                            # (batch_size)
correct_sum =torch.sum(correct, dim=1)
mask_sum == correct_sum


 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.ByteTensor of size 10]

In [ ]:
sampled = []
for sentence in output:
    tmp = ""
    for idx in sentence:
        token = id2word[idx]
        tmp += token
    sampled.append(tmp)